In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import xesmf as xe
from gfdl_grid_fx import grid_model_sym_approx, regrid_regions_gfdl
from other_grid_fx import generate_reg_grid#, regrid_regions_reg
from plot_fx import make_regional_plots
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from nch import compute_error
import os
#import matplotlib

# matplotlib.use("Agg")  # non-X windows backend

#import matplotlib.pyplot as plt
#import numpy as np
import intake
import sys
import yaml

In [ ]:
cat = intake.open_esm_datastore("/glade/work/netige/mdtf_Nov24/data_catalogs/s_ECMWF-IFS-HR_e_hist-1950_vl_r1i1p1f1_f_mon_v_zos.json")
# filter catalog by desired variable and output frequency
zos_subset = cat.search(variable_id=zos_var, frequency="mon")
# examine assets for a specific file
#zos_subset['CMIP.synthetic.day.r1i1p1f1.day.gr.atmos.r1i1p1f1.1980-01-01-1984-12-31'].df
# convert zos_subset catalog to an xarray dataset dict
zos_dict = zos_subset.to_dataset_dict(
    xarray_open_kwargs={"decode_times": True, "use_cftime": True}
)

In [4]:
cat = intake.open_esm_datastore("/glade/work/netige/mdtf_Nov24/data_catalogs/s_ECMWF-IFS-HR_e_hist-1950_vl_r1i1p1f1_f_mon_v_zos.json")


In [6]:
zos_subset = cat.search(variable_id="zos", frequency="mon")

In [7]:
zos_subset

,unique
activity_id,1
branch_method,1
branch_time_in_child,1
branch_time_in_parent,1
experiment,1
experiment_id,1
frequency,1
grid,1
grid_label,1
institution_id,1


In [8]:
zos_dict = zos_subset.to_dataset_dict(
    xarray_open_kwargs={"decode_times": True, "use_cftime": True}
)
zos_dict


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.frequency.member_id.table_id.grid_label.realm.variant_label'


{'HighResMIP.ECMWF.ECMWF-IFS-HR.hist-1950.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1': <xarray.Dataset>
 Dimensions:             (time: 780, bnds: 2, i: 1021, j: 1442, vertices: 4)
 Coordinates:
   * time                (time) object 1950-01-16 12:00:00 ... 2014-12-16 12:0...
     time_bnds           (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
   * i                   (i) int32 1 2 3 4 5 6 ... 1016 1017 1018 1019 1020 1021
   * j                   (j) int32 1 2 3 4 5 6 ... 1437 1438 1439 1440 1441 1442
     latitude            (i, j) float32 dask.array<chunksize=(511, 721), meta=np.ndarray>
     longitude           (i, j) float32 dask.array<chunksize=(511, 721), meta=np.ndarray>
     vertices_latitude   (i, j, vertices) float32 dask.array<chunksize=(511, 721, 1), meta=np.ndarray>
     vertices_longitude  (i, j, vertices) float32 dask.array<chunksize=(511, 721, 1), meta=np.ndarray>
 Dimensions without coordinates: bnds, vertices
 Data variables:
     zos                 (

In [9]:
key = 'HighResMIP.ECMWF.ECMWF-IFS-HR.hist-1950.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1'
dataset = zos_dict[key]

In [10]:
dataset

<xarray.Dataset>
Dimensions:             (time: 780, bnds: 2, i: 1021, j: 1442, vertices: 4)
Coordinates:
  * time                (time) object 1950-01-16 12:00:00 ... 2014-12-16 12:0...
    time_bnds           (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
  * i                   (i) int32 1 2 3 4 5 6 ... 1016 1017 1018 1019 1020 1021
  * j                   (j) int32 1 2 3 4 5 6 ... 1437 1438 1439 1440 1441 1442
    latitude            (i, j) float32 dask.array<chunksize=(511, 721), meta=np.ndarray>
    longitude           (i, j) float32 dask.array<chunksize=(511, 721), meta=np.ndarray>
    vertices_latitude   (i, j, vertices) float32 dask.array<chunksize=(511, 721, 1), meta=np.ndarray>
    vertices_longitude  (i, j, vertices) float32 dask.array<chunksize=(511, 721, 1), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    zos                 (time, i, j) float32 dask.array<chunksize=(1, 511, 721), meta=np.ndarray>
Attributes: (12/83)
    Conventions:                             CF-1.7 CMIP-6.0
    activity_id:                             HighResMIP
    branch_method:                           Initialized directly from parent...
    contact:                                 chris.roberts@ecmwf.int
    end_year:                                2014
    experiment:                              coupled historical 1950-2014
    ...                                      ...
    intake_esm_attrs:long_name:              Sea Surface Height Above Geoid
    intake_esm_attrs:units:                  m
    intake_esm_attrs:vertical_levels:        1
    intake_esm_attrs:version:                v0
    intake_esm_attrs:_data_format_:          netcdf
    intake_esm_dataset_key:                  HighResMIP.ECMWF.ECMWF-IFS-HR.hi...

In [3]:
ds1 = xr.open_dataset('/glade/work/netige/Data/CM4_zos/zos_Omon_GFDL-CM4_historical_r1i1p1f1_gn_199001-200912.nc')
ds2 = xr.open_dataset('/glade/work/netige/Data/CM4_zos/zos_Omon_GFDL-CM4_historical_r1i1p1f1_gn_201001-201412.nc')

In [4]:
ds1

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 240, x: 1440, y: 1080, vertex: 4)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
  * time       (time) object 1990-01-16 12:00:00 ... 2009-12-16 12:00:00
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
    lon        (y, x) float32 ...
    lat        (y, x) float32 ...
Dimensions without coordinates: vertex
Data variables:
    time_bnds  (time, bnds) object ...
    zos        (time, y, x) float32 ...
    lat_bnds   (y, x, vertex) float32 ...
    lon_bnds   (y, x, vertex) float32 ...
Attributes: (12/46)
    title:                  NOAA GFDL GFDL-CM4 model output prepared for CMIP...
    external_variables:     areacello
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               Omon
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            zos
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1
    branch_time_in_parent:  36500.0
    parent_time_units:      days since 0001-1-1

In [5]:
ds2

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 60, x: 1440, y: 1080, vertex: 4)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
  * time       (time) object 2010-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
    lon        (y, x) float32 ...
    lat        (y, x) float32 ...
Dimensions without coordinates: vertex
Data variables:
    time_bnds  (time, bnds) object ...
    zos        (time, y, x) float32 ...
    lat_bnds   (y, x, vertex) float32 ...
    lon_bnds   (y, x, vertex) float32 ...
Attributes: (12/46)
    title:                  NOAA GFDL GFDL-CM4 model output prepared for CMIP...
    external_variables:     areacello
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               Omon
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            zos
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1
    branch_time_in_parent:  36500.0
    parent_time_units:      days since 0001-1-1

In [6]:
# Merge the datasets
merged_ds = xr.merge([ds1, ds2])

In [7]:
merged_ds.time

<xarray.DataArray 'time' (time: 300)>
array([cftime.DatetimeNoLeap(1990, 1, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1990, 2, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1990, 3, 16, 12, 0, 0, 0, has_year_zero=True),
       ...,
       cftime.DatetimeNoLeap(2014, 10, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 11, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 12, 16, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 1990-01-16 12:00:00 ... 2014-12-16 12:00:00
Attributes:
    long_name:      time
    axis:           T
    calendar_type:  noleap
    bounds:         time_bnds
    standard_name:  time
    description:    Temporal mean

In [9]:
merged_ds.to_netcdf("/glade/work/netige/mdtf_Nov24/mdtf/inputdata/model/zos_Omon_GFDL-CM4_historical_r1i1p1f1_gn_199001-201412.nc")


In [10]:
import intake
catalog = intake.open_esm_datastore('/glade/work/netige/mdtf_Nov24/data_catalogs/CM4_zos.json')
print(catalog.search(variable_id='zos'))


<CM4_zos catalog with 1 dataset(s) from 1 asset(s)>


In [11]:
import intake
catalog = intake.open_esm_datastore('/glade/work/netige/mdtf_Nov24/data_catalogs/CM4_zos.json')

# Print available datasets and their metadata
print(catalog)
print(catalog.df)


<CM4_zos catalog with 1 dataset(s) from 1 asset(s)>
  activity_id branch_method  branch_time_in_child  branch_time_in_parent  \
0        CMIP      standard                   0.0                36500.0   

   experiment experiment_id frequency  \
0  historical    historical       mon   

                                                grid grid_label  \
0  ocean data on native grid tripolar - nominal 0...         gn   

  institution_id  ...                   standard_name  \
0      NOAA-GFDL  ...  sea_surface_height_above_geoid   

                        long_name units vertical_levels init_year  \
0  Sea Surface Height Above Geoid     m               1       NaN   

            start_time             end_time  \
0  1990-01-16 12:00:00  2014-12-16 12:00:00   

                                time_range  \
0  1990-01-16 12:00:00-2014-12-16 12:00:00   

                                                path version  
0  /glade/work/netige/mdtf_Nov24/mdtf/inputdata/m...      v2  

[1 rows 

In [12]:
print(catalog)


<CM4_zos catalog with 1 dataset(s) from 1 asset(s)>


In [14]:
catalog.df

,activity_id,branch_method,branch_time_in_child,branch_time_in_parent,experiment,experiment_id,frequency,grid,grid_label,institution_id,...,standard_name,long_name,units,vertical_levels,init_year,start_time,end_time,time_range,path,version
0,CMIP,standard,0.0,36500.0,historical,historical,mon,ocean data on native grid tripolar - nominal 0...,gn,NOAA-GFDL,...,sea_surface_height_above_geoid,Sea Surface Height Above Geoid,m,1,NaN,1990-01-16 12:00:00,2014-12-16 12:00:00,1990-01-16 12:00:00-2014-12-16 12:00:00,/glade/work/netige/mdtf_Nov24/mdtf/inputdata/m...,v2


In [15]:
import xarray as xr

file_path = "/glade/work/netige/mdtf_Nov24/mdtf/inputdata/model/CM4/zos_Omon_GFDL-CM4_historical_r1i1p1f1_gn_199001-201412.nc"
ds = xr.open_dataset(file_path)

# Check for 'zos' in the variables
print("Available variables:", list(ds.variables))
if 'zos' in ds.variables:
    print("Variable 'zos' found!")
else:
    print("Variable 'zos' not found.")


Available variables: ['bnds', 'time', 'x', 'y', 'time_bnds', 'zos', 'lon', 'lat', 'lat_bnds', 'lon_bnds']
Variable 'zos' found!


In [16]:
merged_ds

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 300, x: 1440, y: 1080, vertex: 4)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
  * time       (time) object 1990-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
    lon        (y, x) float32 -299.7 -299.5 -299.2 -299.0 ... 60.0 60.0 60.0
    lat        (y, x) float32 -79.81 -79.81 -79.8 -79.8 ... 64.33 64.22 64.11
Dimensions without coordinates: vertex
Data variables:
    time_bnds  (time, bnds) object 1990-01-01 00:00:00 ... 2015-01-01 00:00:00
    zos        (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    lat_bnds   (y, x, vertex) float32 -79.77 -79.78 -79.84 ... 64.17 64.17 64.06
    lon_bnds   (y, x, vertex) float32 -299.6 -299.8 -299.8 ... 60.0 60.0 60.0
Attributes: (12/46)
    title:                  NOAA GFDL GFDL-CM4 model output prepared for CMIP...
    external_variables:     areacello
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               Omon
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            zos
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1
    branch_time_in_parent:  36500.0
    parent_time_units:      days since 0001-1-1

In [17]:
import intake

# Open the catalog
catalog = intake.open_esm_datastore('/glade/work/netige/mdtf_Nov24/data_catalogs/CM4_zos.json')

# Print all catalog entries
print("Catalog entries:")
print(catalog.df)

# Search for the 'zos' variable in the catalog
subset = catalog.search(variable_id='zos')
print("\nSubset for 'zos':")
print(subset)

# Convert the subset to an xarray Dataset
ds_dict = subset.to_dataset_dict()
for key, ds in ds_dict.items():
    print(f"\nDataset: {key}")
    print(ds)


Catalog entries:
  activity_id branch_method  branch_time_in_child  branch_time_in_parent  \
0        CMIP      standard                   0.0                36500.0   

   experiment experiment_id frequency  \
0  historical    historical       mon   

                                                grid grid_label  \
0  ocean data on native grid tripolar - nominal 0...         gn   

  institution_id  ...                   standard_name  \
0      NOAA-GFDL  ...  sea_surface_height_above_geoid   

                        long_name units vertical_levels init_year  \
0  Sea Surface Height Above Geoid     m               1       NaN   

            start_time             end_time  \
0  1990-01-16 12:00:00  2014-12-16 12:00:00   

                                time_range  \
0  1990-01-16 12:00:00-2014-12-16 12:00:00   

                                                path version  
0  /glade/work/netige/mdtf_Nov24/mdtf/inputdata/m...      v2  

[1 rows x 36 columns]

Subset for 'zos':
<C


Dataset: CMIP.NOAA-GFDL.GFDL-CM4.historical.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1
<xarray.Dataset>
Dimensions:    (bnds: 2, time: 300, x: 1440, y: 1080, vertex: 4)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
  * time       (time) object 1990-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
Dimensions without coordinates: vertex
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(300, 2), meta=np.ndarray>
    zos        (time, y, x) float32 dask.array<chunksize=(300, 1080, 1440), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440,

In [18]:
ls -l /glade/work/netige/mdtf_Nov24/mdtf/inputdata/model/CM4/zos_Omon_GFDL-CM4_historical_r1i1p1f1_gn_199001-201412.nc


-rw-r--r-- 1 netige ncar 1928507661 Jan 13 06:53 /glade/work/netige/mdtf_Nov24/mdtf/inputdata/model/CM4/zos_Omon_GFDL-CM4_historical_r1i1p1f1_gn_199001-201412.nc


In [20]:
ncdump -h /glade/work/netige/mdtf_Nov24/mdtf/inputdata/model/CM4/zos_Omon_GFDL-CM4_historical_r1i1p1f1_gn_199001_201412.nc


NameError: name 'ncdump' is not defined

In [23]:
cat = intake.open_esm_datastore('/glade/work/netige/mdtf_Nov24/data_catalogs/CM4_zos.json')
# filter catalog by desired variable and output frequency
zos_subset = cat.search(variable_id='zos', frequency="mon")
# examine assets for a specific file
#zos_subset['CMIP.synthetic.day.r1i1p1f1.day.gr.atmos.r1i1p1f1.1980-01-01-1984-12-31'].df
# convert zos_subset catalog to an xarray dataset dict
zos_dict = zos_subset.to_dataset_dict(
    xarray_open_kwargs={"decode_times": True, "use_cftime": True}
)

# Extract the dataset from the dictionary
key = 'CMIP.NOAA-GFDL.GFDL-CM4.historical.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1'
dataset = zos_dict[key]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.frequency.member_id.table_id.grid_label.realm.variant_label'


In [27]:
zos_subset

,unique
activity_id,1
branch_method,1
branch_time_in_child,1
branch_time_in_parent,1
experiment,1
experiment_id,1
frequency,1
grid,1
grid_label,1
institution_id,1


In [25]:
zos_dict

{'CMIP.NOAA-GFDL.GFDL-CM4.historical.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1': <xarray.Dataset>
 Dimensions:    (bnds: 2, time: 300, x: 1440, y: 1080, vertex: 4)
 Coordinates:
   * bnds       (bnds) float64 1.0 2.0
   * time       (time) object 1990-01-16 12:00:00 ... 2014-12-16 12:00:00
     time_bnds  (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
   * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
   * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
     lon        (y, x) float32 dask.array<chunksize=(540, 720), meta=np.ndarray>
     lat        (y, x) float32 dask.array<chunksize=(540, 720), meta=np.ndarray>
     lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(540, 720, 1), meta=np.ndarray>
     lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(540, 720, 1), meta=np.ndarray>
 Dimensions without coordinates: vertex
 Data variables:
     zos        (time, y, x) float32 dask.array<chunksize=(1, 540, 720), m

In [2]:
cat = intake.open_esm_datastore('/glade/work/netige/mdtf_Nov24/data_catalogs/ESM4_zos.json')
# filter catalog by desired variable and output frequency
zos_subset = cat.search(variable_id='zos', frequency="mon")
# examine assets for a specific file
#zos_subset['CMIP.synthetic.day.r1i1p1f1.day.gr.atmos.r1i1p1f1.1980-01-01-1984-12-31'].df
# convert zos_subset catalog to an xarray dataset dict
zos_dict = zos_subset.to_dataset_dict(
    xarray_open_kwargs={"decode_times": True, "use_cftime": True}
)

# Extract the dataset from the dictionary
key = 'CMIP.NOAA-GFDL.GFDL-CM4.historical.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1'
dataset = zos_dict[key]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.frequency.member_id.table_id.grid_label.realm.variant_label'


KeyError: 'CMIP.NOAA-GFDL.GFDL-CM4.historical.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1'

In [3]:
zos_dict

{'CMIP.NOAA-GFDL.GFDL-ESM4.historical.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1': <xarray.Dataset>
 Dimensions:    (y: 576, x: 720, vertex: 4, bnds: 2, time: 300)
 Coordinates:
     lat        (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
     lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(288, 360, 2), meta=np.ndarray>
     lon        (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
     lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(288, 360, 2), meta=np.ndarray>
   * bnds       (bnds) float64 1.0 2.0
   * time       (time) object 1990-01-16 12:00:00 ... 2014-12-16 12:00:00
     time_bnds  (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
   * x          (x) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
   * y          (y) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
 Dimensions without coordinates: vertex
 Data variables:
     zos        (time, y, x) float32 dask.array<chunksize=(1, 576, 720), me

In [4]:
print(list(zos_dict.keys()))


['CMIP.NOAA-GFDL.GFDL-ESM4.historical.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1']


In [6]:
zos_dict.get("CMIP.NOAA-GFDL.GFDL-ESM4.historical.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1")

<xarray.Dataset>
Dimensions:    (y: 576, x: 720, vertex: 4, bnds: 2, time: 300)
Coordinates:
    lat        (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(288, 360, 2), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(288, 360, 2), meta=np.ndarray>
  * bnds       (bnds) float64 1.0 2.0
  * time       (time) object 1990-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
  * x          (x) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * y          (y) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Dimensions without coordinates: vertex
Data variables:
    zos        (time, y, x) float32 dask.array<chunksize=(1, 576, 720), meta=np.ndarray>
Attributes: (12/78)
    title:                                   NOAA GFDL GFDL-ESM4 model output...
    external_variables:                      areacello
    history:                                 File was processed by fremetar (...
    table_id:                                Omon
    activity_id:                             CMIP
    branch_method:                           standard
    ...                                      ...
    intake_esm_attrs:long_name:              Sea Surface Height Above Geoid
    intake_esm_attrs:units:                  m
    intake_esm_attrs:vertical_levels:        1
    intake_esm_attrs:version:                v0
    intake_esm_attrs:_data_format_:          netcdf
    intake_esm_dataset_key:                  CMIP.NOAA-GFDL.GFDL-ESM4.histori...